In [1]:
"""
02_data_cleaning.ipynb
======================
Data cleaning pipeline for Strava running data.

Issues identified in exploration:
1. suffer_score: 100% null → DROP
2. workout_type: 57% null → Keep, categorize as 'unspecified'
3. avg_heartrate/max_heartrate: 22% null → Flag, don't impute
4. avg_cadence: 22% null → Flag, don't impute  
5. max_speed_kmh: 2 nulls → Impute from avg_speed
6. Potential outliers: cadence < 60, pace extremes
7. Date columns: Convert to datetime

Output: cleaned dataset saved to data/processed/
"""

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# Paths
PROJECT_ROOT = Path("/Users/venzchenfoo/Documents/All Files/AI and IT/projects/HM_Training_Predictor")
DATA_RAW = PROJECT_ROOT / "data" / "raw"
DATA_PROCESSED = PROJECT_ROOT / "data" / "processed"
FIGURES = PROJECT_ROOT / "reports" / "figures"

# Display settings
pd.set_option('display.max_columns', None)

In [2]:
# Load raw data
df_raw = pd.read_csv(DATA_RAW / "strava_runs_bulk.csv")

print(f"Raw data shape: {df_raw.shape}")
print(f"Columns: {df_raw.columns.tolist()}")

Raw data shape: (256, 16)
Columns: ['id', 'name', 'start_date', 'start_datetime', 'distance_km', 'moving_time_min', 'pace_min_per_km', 'elevation_gain_m', 'avg_speed_kmh', 'max_speed_kmh', 'avg_heartrate', 'max_heartrate', 'avg_cadence', 'suffer_score', 'elapsed_time_min', 'workout_type']


In [3]:
# Track all cleaning operations
cleaning_log = []

def log_cleaning(operation, before_count, after_count, details=""):
    """Log each cleaning step for documentation."""
    cleaning_log.append({
        'operation': operation,
        'rows_before': before_count,
        'rows_after': after_count,
        'rows_affected': before_count - after_count,
        'details': details
    })
    print(f"✓ {operation}: {before_count} → {after_count} ({before_count - after_count} affected)")

# Start with a copy
df = df_raw.copy()
log_cleaning("Load raw data", len(df_raw), len(df), "Initial load")

✓ Load raw data: 256 → 256 (0 affected)


In [4]:
# Step 1: Drop columns with no useful data
# suffer_score is 100% null - Strava's "Relative Effort" requires premium/specific devices

cols_to_drop = ['suffer_score']

before = df.shape[1]
df = df.drop(columns=cols_to_drop)
after = df.shape[1]

log_cleaning(
    "Drop unusable columns", 
    before, 
    after, 
    f"Dropped: {cols_to_drop} (100% null)"
)

print(f"\nRemaining columns: {df.columns.tolist()}")

✓ Drop unusable columns: 16 → 15 (1 affected)

Remaining columns: ['id', 'name', 'start_date', 'start_datetime', 'distance_km', 'moving_time_min', 'pace_min_per_km', 'elevation_gain_m', 'avg_speed_kmh', 'max_speed_kmh', 'avg_heartrate', 'max_heartrate', 'avg_cadence', 'elapsed_time_min', 'workout_type']


In [5]:
# Step 2: Convert date columns to proper datetime types

df['start_date'] = pd.to_datetime(df['start_date'])
df['start_datetime'] = pd.to_datetime(df['start_datetime'])

# Extract useful date components
df['year'] = df['start_date'].dt.year
df['month'] = df['start_date'].dt.month
df['week'] = df['start_date'].dt.isocalendar().week
df['day_of_week'] = df['start_date'].dt.dayofweek  # 0=Monday, 6=Sunday
df['day_name'] = df['start_date'].dt.day_name()

log_cleaning(
    "Convert date columns",
    len(df),
    len(df),
    "Added: year, month, week, day_of_week, day_name"
)

print(df[['start_date', 'year', 'month', 'week', 'day_name']].head())

✓ Convert date columns: 256 → 256 (0 affected)
  start_date  year  month  week  day_name
0 2025-11-10  2025     11    46    Monday
1 2025-09-21  2025      9    38    Sunday
2 2025-09-18  2025      9    38  Thursday
3 2025-09-12  2025      9    37    Friday
4 2025-09-09  2025      9    37   Tuesday


In [6]:
# Step 3: Clean workout_type column
# Strava workout types: 0=default, 1=race, 2=long run, 3=workout/intervals

print("workout_type value counts (before):")
print(df['workout_type'].value_counts(dropna=False))

# Map numeric codes to readable labels
workout_mapping = {
    0.0: 'default',
    1.0: 'race',
    2.0: 'long_run', 
    3.0: 'workout'
}

df['workout_type_clean'] = df['workout_type'].map(workout_mapping).fillna('workout')

log_cleaning(
    "Clean workout_type",
    len(df),
    len(df),
    "Mapped codes to labels, NaN → 'workout'"
)

print("\nworkout_type_clean value counts (after):")
print(df['workout_type_clean'].value_counts())

workout_type value counts (before):
workout_type
NaN    147
3.0     81
2.0     25
1.0      3
Name: count, dtype: int64
✓ Clean workout_type: 256 → 256 (0 affected)

workout_type_clean value counts (after):
workout_type_clean
workout     228
long_run     25
race          3
Name: count, dtype: int64


In [7]:
# Step 4: Create flags for missing sensor data
# Don't impute HR/cadence - absence indicates no device, not zero

df['has_heartrate'] = df['avg_heartrate'].notna().astype(int)
df['has_cadence'] = df['avg_cadence'].notna().astype(int)

log_cleaning(
    "Flag missing sensor data",
    len(df),
    len(df),
    f"has_heartrate: {df['has_heartrate'].sum()}/{ len(df)}, has_cadence: {df['has_cadence'].sum()}/{len(df)}"
)

# Summary
print("\nSensor data availability:")
print(f"  Runs with HR data: {df['has_heartrate'].sum()} ({df['has_heartrate'].mean()*100:.1f}%)")
print(f"  Runs with cadence: {df['has_cadence'].sum()} ({df['has_cadence'].mean()*100:.1f}%)")

✓ Flag missing sensor data: 256 → 256 (0 affected)

Sensor data availability:
  Runs with HR data: 200 (78.1%)
  Runs with cadence: 199 (77.7%)


In [8]:
# Step 5: Impute missing max_speed_kmh (only 2 values)
# Use ratio from runs that have both values

mask_missing = df['max_speed_kmh'].isna()
print(f"Runs missing max_speed_kmh: {mask_missing.sum()}")

# Calculate typical ratio of max_speed to avg_speed
valid_speeds = df[df['max_speed_kmh'].notna()]
speed_ratio = (valid_speeds['max_speed_kmh'] / valid_speeds['avg_speed_kmh']).median()
print(f"Median max/avg speed ratio: {speed_ratio:.2f}")

# Impute
df.loc[mask_missing, 'max_speed_kmh'] = df.loc[mask_missing, 'avg_speed_kmh'] * speed_ratio

log_cleaning(
    "Impute max_speed_kmh",
    mask_missing.sum(),
    0,
    f"Used median ratio ({speed_ratio:.2f}) × avg_speed"
)

Runs missing max_speed_kmh: 2
Median max/avg speed ratio: 1.60
✓ Impute max_speed_kmh: 2 → 0 (2 affected)


In [9]:
# Step 6: Identify and flag outliers (don't remove - flag for review)

def flag_outliers_iqr(series, multiplier=1.5):
    """Return boolean mask of outliers using IQR method."""
    Q1, Q3 = series.quantile([0.25, 0.75])
    IQR = Q3 - Q1
    lower, upper = Q1 - multiplier * IQR, Q3 + multiplier * IQR
    return (series < lower) | (series > upper)

# Flag outliers in key metrics
df['outlier_pace'] = flag_outliers_iqr(df['pace_min_per_km'])
df['outlier_cadence'] = flag_outliers_iqr(df['avg_cadence'].dropna()).reindex(df.index, fill_value=False)
df['outlier_distance'] = flag_outliers_iqr(df['distance_km'])

# Combined outlier flag
df['is_outlier'] = df['outlier_pace'] | df['outlier_cadence'] | df['outlier_distance']

outlier_count = df['is_outlier'].sum()
log_cleaning(
    "Flag outliers",
    len(df),
    len(df),
    f"Flagged {outlier_count} runs as potential outliers"
)

print("\nOutlier breakdown:")
print(f"  Pace outliers: {df['outlier_pace'].sum()}")
print(f"  Cadence outliers: {df['outlier_cadence'].sum()}")
print(f"  Distance outliers: {df['outlier_distance'].sum()}")

✓ Flag outliers: 256 → 256 (0 affected)

Outlier breakdown:
  Pace outliers: 9
  Cadence outliers: 5
  Distance outliers: 19


In [14]:
# Step 7: Review flagged outliers before deciding action

print("=== Flagged Outliers ===\n")

outlier_runs = df[df['is_outlier']][
    ['start_date', 'name', 'distance_km', 'pace_min_per_km', 
     'avg_cadence', 'workout_type_clean', 'outlier_pace', 'outlier_cadence', 'outlier_distance']
].sort_values('start_date')

print(outlier_runs.to_string())

=== Flagged Outliers ===

    start_date                                              name  distance_km  pace_min_per_km  avg_cadence workout_type_clean  outlier_pace  outlier_cadence  outlier_distance
255 2023-05-26                                     First Run 1.0         2.97             7.67          NaN            workout          True            False             False
252 2023-05-27                                    Second Run 2.0         2.30             8.58          NaN            workout          True            False             False
159 2024-04-07                                                17        17.30             6.07         82.7            workout         False            False              True
156 2024-04-12                                          Long Run        19.12             5.78         82.7            workout         False            False              True
137 2024-05-19                                  Morning Long Run        17.61             6.17

In [17]:
# Cell 10b - Refine outlier classification

# Distance outliers are actually LONG RUNS - valuable, not errors
# Only flag true anomalies: sensor errors or data quality issues

# Suspicious cadence (likely sensor error or significant walking)
df['suspicious_cadence'] = (df['avg_cadence'] < 60) & df['avg_cadence'].notna()

# Suspicious pace (> 10 min/km likely has significant walking or GPS issues)
df['suspicious_pace'] = df['pace_min_per_km'] > 10

# Reclassify: true anomaly vs legitimate outlier
df['is_anomaly'] = df['suspicious_cadence'] | df['suspicious_pace']
df['is_long_run'] = df['distance_km'] >= 15

# Update is_outlier to only flag true anomalies
df['is_outlier'] = df['is_anomaly']

print("=== Refined Classification ===")
print(f"Long runs (≥15km): {df['is_long_run'].sum()} - KEEP for modeling")
print(f"True anomalies: {df['is_anomaly'].sum()} - Review/exclude from model")

print("\n=== Anomalous Runs ===")
anomalies = df[df['is_anomaly']][
    ['start_date', 'name', 'distance_km', 'pace_min_per_km', 'avg_cadence']
]
print(anomalies.to_string())

=== Refined Classification ===
Long runs (≥15km): 24 - KEEP for modeling
True anomalies: 1 - Review/exclude from model

=== Anomalous Runs ===
   start_date       name  distance_km  pace_min_per_km  avg_cadence
29 2025-05-29  Lunch Run         5.09              9.9         41.0


In [19]:
# Step 8: Add derived metrics useful for analysis

# Rest time (difference between elapsed and moving time)
df['rest_time_min'] = df['elapsed_time_min'] - df['moving_time_min']
df['rest_pct'] = (df['rest_time_min'] / df['elapsed_time_min'] * 100).round(2)

# Pace in seconds (useful for calculations)
df['pace_sec_per_km'] = df['pace_min_per_km'] * 60

# Speed from pace (redundant check)
df['calc_speed_kmh'] = 60 / df['pace_min_per_km']

log_cleaning(
    "Add derived metrics",
    len(df),
    len(df),
    "Added: rest_time_min, rest_pct, pace_sec_per_km"
)

✓ Add derived metrics: 256 → 256 (0 affected)


In [18]:
# Step 9: Select and order final columns

columns_final = [
    # Identifiers
    'id', 'name', 'start_date', 'start_datetime',
    
    # Date components
    'year', 'month', 'week', 'day_of_week', 'day_name',
    
    # Core metrics
    'distance_km', 'moving_time_min', 'elapsed_time_min',
    'pace_min_per_km', 'pace_sec_per_km',
    
    # Speed
    'avg_speed_kmh', 'max_speed_kmh',
    
    # Elevation
    'elevation_gain_m',
    
    # Heart rate (may be null)
    'avg_heartrate', 'max_heartrate',
    
    # Cadence (may be null)
    'avg_cadence',
    
    # Derived
    'rest_time_min', 'rest_pct',
    
    # Categorical
    'workout_type_clean',
    
    # Flags
    'has_heartrate', 'has_cadence', 'is_outlier', 'is_anomaly', 'is_long_run',
    'suspicious_cadence', 'suspicious_pace'
]

df_clean = df[columns_final].copy()

log_cleaning(
    "Final column selection",
    df.shape[1],
    df_clean.shape[1],
    f"Selected {len(columns_final)} columns"
)

print(f"\nFinal columns ({len(df_clean.columns)}):")
print(df_clean.columns.tolist())

✓ Final column selection: 35 → 30 (5 affected)

Final columns (30):
['id', 'name', 'start_date', 'start_datetime', 'year', 'month', 'week', 'day_of_week', 'day_name', 'distance_km', 'moving_time_min', 'elapsed_time_min', 'pace_min_per_km', 'pace_sec_per_km', 'avg_speed_kmh', 'max_speed_kmh', 'elevation_gain_m', 'avg_heartrate', 'max_heartrate', 'avg_cadence', 'rest_time_min', 'rest_pct', 'workout_type_clean', 'has_heartrate', 'has_cadence', 'is_outlier', 'is_anomaly', 'is_long_run', 'suspicious_cadence', 'suspicious_pace']


In [20]:
# Cleaning summary report

print("=" * 50)
print("CLEANING SUMMARY")
print("=" * 50)

print(f"\nDataset shape: {df_clean.shape[0]} rows × {df_clean.shape[1]} columns")
print(f"Date range: {df_clean['start_date'].min().date()} to {df_clean['start_date'].max().date()}")

print("\n--- Cleaning Log ---")
cleaning_df = pd.DataFrame(cleaning_log)
print(cleaning_df.to_string(index=False))

print("\n--- Null Values Remaining ---")
nulls = df_clean.isnull().sum()
nulls_remaining = nulls[nulls > 0]
if len(nulls_remaining) > 0:
    print(nulls_remaining)
else:
    print("No null values (except expected HR/cadence)")

print("\n--- Data Types ---")
print(df_clean.dtypes)

CLEANING SUMMARY

Dataset shape: 256 rows × 30 columns
Date range: 2023-05-26 to 2025-11-10

--- Cleaning Log ---
               operation  rows_before  rows_after  rows_affected                                         details
           Load raw data          256         256              0                                    Initial load
   Drop unusable columns           16          15              1           Dropped: ['suffer_score'] (100% null)
    Convert date columns          256         256              0 Added: year, month, week, day_of_week, day_name
      Clean workout_type          256         256              0         Mapped codes to labels, NaN → 'workout'
Flag missing sensor data          256         256              0    has_heartrate: 200/256, has_cadence: 199/256
    Impute max_speed_kmh            2           0              2            Used median ratio (1.60) × avg_speed
           Flag outliers          256         256              0           Flagged 28 runs as p

In [21]:
# Save cleaned dataset

output_path = DATA_PROCESSED / "strava_runs_cleaned.csv"
df_clean.to_csv(output_path, index=False)

print(f"✓ Saved cleaned data to: {output_path}")

# Quick verification
df_verify = pd.read_csv(output_path)
print(f"\nVerification - reloaded shape: {df_verify.shape}")

✓ Saved cleaned data to: /Users/venzchenfoo/Documents/All Files/AI and IT/projects/HM_Training_Predictor/data/processed/strava_runs_cleaned.csv

Verification - reloaded shape: (256, 30)


In [22]:
# Preview cleaned data

df_clean.head(10)

,id,name,start_date,start_datetime,year,month,week,day_of_week,day_name,distance_km,moving_time_min,elapsed_time_min,pace_min_per_km,pace_sec_per_km,avg_speed_kmh,max_speed_kmh,elevation_gain_m,avg_heartrate,max_heartrate,avg_cadence,rest_time_min,rest_pct,workout_type_clean,has_heartrate,has_cadence,is_outlier,is_anomaly,is_long_run,suspicious_cadence,suspicious_pace
0,16662765870,Morning Run,2025-11-10,2025-11-10 10:17:59+00:00,2025,11,46,0,Monday,5.01,31.33,31.62,6.26,375.6,9.59,18.94,31.0,135.4,183.0,81.8,0.29,0.92,workout,1,1,False,False,False,False,False
1,15885426834,PSD Halb Marathon,2025-09-21,2025-09-21 09:59:57+00:00,2025,9,38,6,Sunday,21.15,109.98,109.98,5.20,312.0,11.54,18.65,54.0,178.6,198.0,82.4,0.00,0.00,race,1,1,False,False,True,False,False
2,15874728860,Training Day 28 (Final Run),2025-09-18,2025-09-18 18:10:35+00:00,2025,9,38,3,Thursday,5.02,27.27,28.03,5.44,326.4,11.04,20.30,29.0,159.1,178.0,82.8,0.76,2.71,workout,1,1,False,False,False,False,False
3,15784733272,Training Day 27 (5KM),2025-09-12,2025-09-12 08:03:10+00:00,2025,9,37,4,Friday,5.01,26.87,27.55,5.36,321.6,11.19,19.04,33.0,150.8,164.0,83.6,0.68,2.47,workout,1,1,False,False,False,False,False
4,15753444812,Training Day 26 (7KM Easy),2025-09-09,2025-09-09 11:49:11+00:00,2025,9,37,1,Tuesday,7.30,42.27,43.88,5.79,347.4,10.36,16.85,45.0,155.4,172.0,81.9,1.61,3.67,workout,1,1,False,False,False,False,False
5,15724236897,Training Day 25 (15KM Long Run),2025-09-07,2025-09-07 06:18:56+00:00,2025,9,36,6,Sunday,15.65,102.65,102.82,6.56,393.6,9.15,14.55,70.0,147.8,176.0,81.8,0.17,0.17,long_run,1,1,False,False,True,False,False
6,15696345469,Afternoon Run,2025-09-04,2025-09-04 17:27:59+00:00,2025,9,36,3,Thursday,10.02,60.53,61.33,6.04,362.4,9.94,16.78,50.0,153.5,188.0,81.5,0.80,1.30,workout,1,1,False,False,False,False,False
7,15669973087,Training Day 23 (5KM easy),2025-09-02,2025-09-02 12:21:21+00:00,2025,9,36,1,Tuesday,5.02,27.40,27.83,5.46,327.6,10.99,19.13,31.0,151.8,182.0,82.0,0.43,1.55,workout,1,1,False,False,False,False,False
8,15625627662,Afternoon Run,2025-08-27,2025-08-27 17:44:01+00:00,2025,8,35,2,Wednesday,7.34,43.57,44.33,5.94,356.4,10.11,16.08,30.0,155.3,184.0,81.7,0.76,1.71,workout,1,1,False,False,False,False,False
9,15625627650,Morning Run,2025-08-20,2025-08-20 06:29:05+00:00,2025,8,34,2,Wednesday,5.03,28.28,28.78,5.62,337.2,10.67,17.85,32.0,151.1,171.0,82.0,0.50,1.74,workout,1,1,False,False,False,False,False
